# 训练预测
## 1. 单模型预测 
我们将刚才固化下来的InceptionV3, InceptionResNetV2, ResNet50模型载入, 预测测试集, 并生成要向Kaggle提交的.csv文件

### 载入必要的库

In [1]:
from keras.models import *
from keras.preprocessing.image import *
from keras.applications import *

import pandas as pd

/home/autel/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### 定义预测函数, 每个模型生成相应的预测文件

In [2]:
def predict_cat_dog(model_name, img_size, preprocess_func):
    batch_size = 100
    
    model = load_model(model_name)

    gen = ImageDataGenerator(preprocessing_function=preprocess_func)
    test_generator = gen.flow_from_directory("./Dataset/test2", img_size, shuffle=False,
                                             batch_size=batch_size, class_mode=None)

    y_test = model.predict_generator(test_generator, test_generator.samples // batch_size)
    y_test = y_test.clip(min=0.005, max=0.995)

    # 产生submission
    df = pd.read_csv("sample_submission.csv")

    for i, fname in enumerate(test_generator.filenames):
        index = int(fname[fname.rfind('/') + 1:fname.rfind('.')])
        df.set_value(index - 1, 'label', y_test[i])

    df.to_csv(model_name.split('_')[0]+'_pred.csv', index=None)

### 分别执行三个模型的预测, 得到三个预测文件

In [3]:
predict_cat_dog('InceptionV3_cvd.h5', (299, 299), inception_v3.preprocess_input)
predict_cat_dog('Xception_cvd.h5', (299, 299), xception.preprocess_input)
predict_cat_dog('InceptionResNetV2_cvd.h5', (299, 299), inception_resnet_v2.preprocess_input)

Found 12500 images belonging to 1 classes.


/home/autel/.local/lib/python3.5/site-packages/ipykernel_launcher.py:18: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


Found 12500 images belonging to 1 classes.
Found 12500 images belonging to 1 classes.


## 2. 融合模型预测
将在测试集图像经过三个模型得到的特征从保存的特征文件中解析出来, 用固化的融合模型进行预测, 生成相应的预测文件

In [4]:
X_train = []
X_test = []
y_pred = []

for filename in ["gap_InceptionV3.h5", "gap_Xception.h5", "gap_InceptionResNetV2.h5"]:
    with h5py.File(filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])

X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)

# 获取保存的模型
model = load_model('model_concat_cvd.h5')

y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)


# 产生submission
df = pd.read_csv("sample_submission.csv")

gen = ImageDataGenerator()
test_generator = gen.flow_from_directory("./Dataset/test2", (224, 224), shuffle=False,
                                         batch_size=16, class_mode=None)

for i, fname in enumerate(test_generator.filenames):
    index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    df.set_value(index-1, 'label', y_pred[i])

df.to_csv('model_concat_pred.csv', index=None)
df.head(10)

12500/12500 [==============================] - 2s 185us/step
Found 12500 images belonging to 1 classes.


/home/autel/.local/lib/python3.5/site-packages/ipykernel_launcher.py:30: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


,id,label
0,1,0.995
1,2,0.995
2,3,0.995
3,4,0.995
4,5,0.005
5,6,0.005
6,7,0.005
7,8,0.005
8,9,0.005
9,10,0.005
